# Helm — Kubernetes Package Manager

Helm is the de facto package manager for Kubernetes. It simplifies deploying and managing complex applications by packaging Kubernetes manifests into reusable **charts**.

## Core concepts

| Concept | Description |
| --- | --- |
| Chart | A package containing Kubernetes resource definitions |
| Release | An instance of a chart deployed to a cluster |
| Repository | A collection of charts (like npm registry) |
| Values | Configuration parameters passed to a chart |
| Template | Go templates that generate Kubernetes manifests |

## Chart structure

```
mychart/
├── Chart.yaml          # Chart metadata (name, version, dependencies)
├── values.yaml         # Default configuration values
├── charts/             # Dependency charts
├── templates/          # Kubernetes manifest templates
│   ├── deployment.yaml
│   ├── service.yaml
│   ├── ingress.yaml
│   ├── _helpers.tpl    # Template helpers/partials
│   └── NOTES.txt       # Post-install instructions
└── .helmignore         # Files to ignore when packaging
```

## Helm CLI commands

```bash
# Add a chart repository
helm repo add bitnami https://charts.bitnami.com/bitnami
helm repo update

# Search for charts
helm search repo nginx
helm search hub prometheus  # Search Artifact Hub

# Install a chart (create a release)
helm install my-nginx bitnami/nginx
helm install my-nginx bitnami/nginx --namespace web --create-namespace

# Install with custom values
helm install my-nginx bitnami/nginx -f custom-values.yaml
helm install my-nginx bitnami/nginx --set replicaCount=3

# List releases
helm list
helm list --all-namespaces

# Upgrade a release
helm upgrade my-nginx bitnami/nginx --set replicaCount=5

# Rollback to previous version
helm rollback my-nginx 1

# Uninstall a release
helm uninstall my-nginx

# View release history
helm history my-nginx
```

## Templating basics

Helm uses Go templates with Sprig functions. Common patterns:

```yaml
# templates/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: {{ include "mychart.fullname" . }}
  labels:
    {{- include "mychart.labels" . | nindent 4 }}
spec:
  replicas: {{ .Values.replicaCount }}
  selector:
    matchLabels:
      {{- include "mychart.selectorLabels" . | nindent 6 }}
  template:
    metadata:
      labels:
        {{- include "mychart.selectorLabels" . | nindent 8 }}
    spec:
      containers:
        - name: {{ .Chart.Name }}
          image: "{{ .Values.image.repository }}:{{ .Values.image.tag | default .Chart.AppVersion }}"
          ports:
            - containerPort: {{ .Values.service.port }}
          {{- if .Values.resources }}
          resources:
            {{- toYaml .Values.resources | nindent 12 }}
          {{- end }}
```

**Template functions:**
- `{{ .Values.x }}` — Access values
- `{{ .Release.Name }}` — Release name
- `{{ .Chart.Name }}` — Chart name
- `{{ include "name" . }}` — Include named template
- `{{ toYaml .Values.x | nindent 4 }}` — Convert to YAML with indentation
- `{{ default "value" .Values.x }}` — Default value
- `{{ if .Values.x }}...{{ end }}` — Conditionals
- `{{ range .Values.list }}...{{ end }}` — Loops

## values.yaml example

```yaml
# values.yaml
replicaCount: 2

image:
  repository: nginx
  tag: "1.25"
  pullPolicy: IfNotPresent

service:
  type: ClusterIP
  port: 80

ingress:
  enabled: true
  className: nginx
  hosts:
    - host: myapp.example.com
      paths:
        - path: /
          pathType: Prefix

resources:
  limits:
    cpu: 100m
    memory: 128Mi
  requests:
    cpu: 50m
    memory: 64Mi

autoscaling:
  enabled: false
  minReplicas: 1
  maxReplicas: 10
  targetCPUUtilizationPercentage: 80
```

## Creating a new chart

```bash
# Create chart scaffold
helm create mychart

# Validate chart syntax
helm lint mychart

# Render templates locally (debug)
helm template my-release mychart
helm template my-release mychart --debug

# Dry-run install
helm install my-release mychart --dry-run

# Package chart for distribution
helm package mychart
# Creates mychart-0.1.0.tgz
```

## Mini example: Chart dependency visualization

In [ ]:
import pandas as pd
import plotly.express as px

# Typical Helm chart adoption
charts = pd.DataFrame({
    "chart": ["nginx-ingress", "prometheus", "grafana", "cert-manager", "redis", "postgresql"],
    "downloads_k": [850, 720, 680, 620, 550, 480],
    "category": ["Ingress", "Monitoring", "Monitoring", "Security", "Cache", "Database"]
})

fig = px.bar(
    charts, x="chart", y="downloads_k", color="category",
    title="Popular Helm charts by downloads (illustrative)",
    labels={"downloads_k": "Downloads (thousands)", "chart": "Chart"}
)
fig

## Best practices

1. **Version everything** — Use semantic versioning for charts
2. **Use `helm diff`** — Preview changes before upgrade (`helm plugin install https://github.com/databus23/helm-diff`)
3. **Keep values.yaml documented** — Add comments explaining each value
4. **Use subcharts for dependencies** — Define in `Chart.yaml`
5. **Test charts** — Use `helm test` with test pods
6. **Store charts in OCI registries** — Modern approach for chart distribution